In [13]:
%reset

In [28]:
import sys; sys.path.insert(1, '../')
import numpy as np
import matplotlib.pyplot as plt
import mne
from actionpy.prepro import map_channels, raw_to_epochs, get_data, preprocessing
%load_ext autoreload
%autoreload 2

The autoreload module is not an IPython extension.


UsageError: Line magic function `%autoreload` not found.


In [3]:
subject = 'S00'

base_path ='C:\\Users\\Marianne\\Desktop\\SCP-ACT\\raw_data\\' + '\\' + subject + '\\' + subject + '_'
files = ['rest_01', 'rest_02', 'exp_01', 'drmt_01']

ending = '.vhdr'

raws = []
for file in files:
    pth_raw = base_path + file + ending
    print(pth_raw)
    raws.append( preprocessing(pth_raw) ) 

C:\Users\Marianne\Desktop\SCP-ACT\raw_data\\S00\S00_rest_01.vhdr


c:\Users\Marianne\Desktop\SCP-ACT\data_analysis\actionpy\prepro\prepro.py:14: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(pth, preload = True, verbose = 0)
c:\Users\Marianne\Desktop\SCP-ACT\data_analysis\actionpy\prepro\prepro.py:14: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(pth, preload = True, verbose = 0)
c:\Users\Marianne\Desktop\SCP-ACT\data_analysis\actionpy\prepro\prepro.py:14: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (500.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(pth, preload = True, verbose = 0)
c:\Users\Marianne\Desktop\SCP-ACT\data_analysis\actionpy\prepro\prepro.py:17: RuntimeWarning: DigMontage is only a subset of info. There are 1 channel position not present in the DigMontage. T

Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


[Parallel(n_jobs=4)]: Done  32 out of  32 | elapsed:    9.3s finished
c:\Users\Marianne\Desktop\SCP-ACT\data_analysis\actionpy\prepro\prepro.py:33: RuntimeWarning: The unit for channel(s) VEOG has changed from C to V.
  raw.set_channel_types(mapping)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  7.0min
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:  7.0min finished
C:\Users\Marianne\anaconda3\envs\py_eeg\lib\site-packages\autoreject\ransac.py:226: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  epochs.interpolate_bads(reset_bads=True)


In [4]:
from actionpy.prepro import get_data
%matplotlib qt
markers = [[2, 4], [2, 4], [2, 6], [None, 4]]
picks = ['Cz', 'Resp']

datas = []
for raw, marker in zip(raws, markers):
    # raw.plot(duration=60, scalings=dict(eeg=120e-6))
    datas.append( get_data(raw.copy().filter(None, 1, n_jobs=-1).resample(100), marker[0], marker[1], picks) )
    # datas is now a list with the cut and filtered data from channels Cz and Resp

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:    3.5s
[Parallel(n_jobs=4)]: Done  31 out of  31 | elapsed:    4.1s finished


Used Annotations descriptions: ['New Segment/', 'Stimulus/S  2', 'Stimulus/S  4']


In [5]:
plt.figure()
ylim = (-9e-5, 9e-5)
refreq = 100
n = len(datas)
for i in range(n):
    num = n*100 + 10 + i+1
    plt.subplot(num)
    plt.plot(raws[i].copy().resample(refreq).times[:len(datas[i][0, :])], np.std(datas[i][:, :], axis=0))


### Global Descriptors

In [19]:
### skip ###

from actionpy.analysis import run_length_encoding, omega_descriptor, sigma_descriptor, phi_descriptor, average_phase_synchrony

metrics = (omega_descriptor, sigma_descriptor, phi_descriptor, average_phase_synchrony)
metrics_names = ('omega', 'sigma', 'phi', 'phase_synchrony')
 results = {channel: dict() for channel in channels}
results = dict()
# for i, channel in enumerate(channels):

for metric, name in zip(metrics, metrics_names):
    results[name] = [[np.real(metric(portion, sfreq=100)) if name=='phi' else np.real(metric(portion)) \
                        for portion in np.array_split(data, n, axis=1)] 
                        for data in datas]
    




ImportError: cannot import name 'sampleentropy' from 'actionpy.analysis' (c:\Users\Marianne\Desktop\SCP-ACT\data_analysis\actionpy\analysis\__init__.py)

In [8]:
### skip ###

import seaborn as sns

plt.figure(figsize=(4, 9))
ylim = (-9e-5, 9e-5)
refreq = 100
n = len(metrics)
for i in range(n):
    num = n*100 + 10 + i+1
    plt.subplot(num)
    sns.boxplot(data=results[metrics_names[i]])
    plt.xticks(ticks=np.arange(len(files)), labels=files)
    plt.title(metrics_names[i])


plt.tight_layout()

In [7]:
### skip ###

from actionpy.analysis import run_length_encoding

run_length_encoding(datas[0][0], gridsize=1, plotme=True)
np.array_split(datas[0][0])

[]


243.35260115606937

In [17]:
from nolds import sampen

n = 14     # wir teilen jeden Datensatz in 14 Teile, sodass wir Teile von 30s bekommen. Damit können wir Statistik für jede Person machen

results = dict()
metrics = (run_length_encoding, sampen, np.std)   # np.std for rms
metric_names = ('run length', 'complexity', 'rms')


for metric, metric_name in zip(metrics, metric_names):
    results[metric_name] = []
    for condition in range(len(datas)):
        pot = []      # we will put all results (for every metric, condition and 30s part) into list "pot"
        
        for i in range(n):     # that way, we actually split the data for every condition and metric into 30s parts
            if metric_name == 'run length':
                a = metric(np.array_split(datas[condition][0], n)[i], gridsize=1)
            else:
                a = metric(np.array_split(datas[condition][0], n)[i])
            pot.append(a)

        results[metric_name].append(pot)    # here, the results saved in the pot are put into the results dict, at the keys metric_name (which we loop through) so we have a dict with data for every metric



[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [29]:
import seaborn as sns
#metric_name = 'run length'
plt.figure()
#sns.violinplot(data=results[metric_name])
#plt.title(metric_name)

plt.subplot(3,1,1)

sns.violinplot(data=results['run length'])
plt.title('run length')

plt.subplot(3,1,2)
sns.violinplot(data=results['complexity'])
plt.title('complexity')

plt.subplot(3,1,3)
sns.violinplot(data=results['rms'])
plt.title('complexity')





Text(0.5, 1.0, 'complexity')